In [11]:
import requests
import json

category_attractions = "attraction"
category_poi = "poi"
location = ""  # Update this with your desired location
name = ""      # Update this with your desired name

# Define URLs for attractions and POIs
url_attractions = f"http://tour-pedia.org/api/getPlaces?category={category_attractions}&location={location}&name={name}"
url_attraction_reviews = f"http://tour-pedia.org/api/getReviews?category={category_attractions}&location={location}&name={name}"
url_poi = f"http://tour-pedia.org/api/getPlaces?category={category_poi}&location={location}&name={name}"
url_poi_reviews = f"http://tour-pedia.org/api/getReviews?category={category_poi}&location={location}&name={name}"

headers = {"accept": "application/json"}

# Function to send requests and save responses to JSON files
def fetch_and_save(url, filename):
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Check if the content is not empty
        if response.text.strip():  # Check if the response has content
            try:
                data = response.json()
                with open(filename, 'w') as json_file:
                    json.dump(data, json_file, indent=4)  # Pretty-print with indent=4
                print(f"Data has been saved to {filename}")
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}. Response content: {response.text[:100]}")
        else:
            print(f"Empty response from {url}. No data saved.")
    else:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}, Response: {response.text[:100]}")

# Fetch and save data for attractions and POIs
fetch_and_save(url_attractions, 'attractions.json')
fetch_and_save(url_attraction_reviews, 'attraction_reviews.json')
fetch_and_save(url_poi, 'poi.json')
fetch_and_save(url_poi_reviews, 'poi_reviews.json')


Data has been saved to attractions.json
Data has been saved to attraction_reviews.json
Data has been saved to poi.json
Data has been saved to poi_reviews.json


In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import json

# Open the file and load the JSON content
with open('merged_attractions_pretty.json', 'r') as file:
   attractions = json.load(file)


# send json file response to mongoDB
uri = "mongodb+srv://////////:///////@/////.////.mongodb.net/test?retryWrites=true&w=majority&tlsAllowInvalidCertificates=true"


# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))


# Change the connection string if needed
db = client['tourpedia_data']  # Create/use a database called 'tourpedia_data'
collection_attractions = db['attractions']  # Create/use a collection called 'restaurants'


# Deletes all documents in the 'restaurants' collection
collection_attractions.delete_many({})

# Inserts all documents
collection_attractions.insert_many(attractions)


print("Data successfully inserted into MongoDB!")


Data successfully inserted into MongoDB!
